In [2]:
# Name: Taylor D. Gabatino
import re

# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [3]:
### Write your code here
import pyspark
from pyspark import SparkContext

In [4]:
sc = SparkContext()

In [116]:
# Version Checking
print(f"Spark version is {sc.version}")

print(f"Phthon version is {sc.pythonVer}")

print(f"The name of the master is {sc.master}")

Spark version is 3.2.0
Phthon version is 3.8
The name of the master is local[*]


### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [5]:
### Write your code here
book_43 = sc.textFile("data/43.txt")

In [6]:
# Check to see that book_43 is of type MapPartitionsRDD
type(book_43)

pyspark.rdd.RDD

In [7]:
# Check to see that this returns True
str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

True

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [8]:
### Write your code here
# The total number of lines in the text file
print(f"Number of lines: {book_43.count()}")

Number of lines: 2935


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [9]:
### Write your code here
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

test_line = "This is an example of that contains 234 and a dash-containing number"

result = clean_split_line(test_line)
result

['THIS',
 'IS',
 'AN',
 'EXAMPLE',
 'OF',
 'THAT',
 'CONTAINS',
 'AND',
 'A',
 'DASH',
 'CONTAINING',
 'NUMBER']

### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [10]:
### Write your code here
words = book_43.flatMap(clean_split_line)
print(f"Number of words in the book: {words.count()}")

Number of words in the book: 29116


### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [11]:
### Write your code here
unique_words = words.map(lambda x: x.lower()).distinct()
print(f"Number of distinct in the book: {unique_words.count()}")

Number of distinct in the book: 4296


### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [95]:
### Write your code here
book_43_counts = book_43.flatMap(clean_split_line).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
book_43_counts.take(1)

[('GUTENBERG', 98)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [96]:
### Write your code here
book_43_counts.map(lambda x: (x[1], x[0])).sortByKey(False).take(20)

[(1807, 'THE'),
 (1068, 'OF'),
 (1043, 'AND'),
 (726, 'TO'),
 (686, 'A'),
 (646, 'I'),
 (485, 'IN'),
 (471, 'WAS'),
 (392, 'THAT'),
 (384, 'HE'),
 (378, 'IT'),
 (312, 'YOU'),
 (308, 'MY'),
 (301, 'WITH'),
 (285, 'HIS'),
 (244, 'HAD'),
 (203, 'AS'),
 (202, 'FOR'),
 (195, 'THIS'),
 (193, 'BUT')]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [97]:
### Write your code here
from sklearn.feature_extraction import _stop_words
list_stops_words = list(_stop_words.ENGLISH_STOP_WORDS)

In [98]:
# Check to see what type and that the list is extracted
type(list_stops_words)

list

In [99]:
# Print out the list
list_stops_words[:10]

['hereafter',
 'amongst',
 'will',
 'whereas',
 'nowhere',
 'being',
 'no',
 'have',
 'neither',
 'fifteen']

### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [105]:
### Write your code here
book_43_counts_filtered = book_43_counts.filter(lambda x: x[0] not in list_stops_words)
type(book_43_counts_filtered)

pyspark.rdd.PipelinedRDD

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [115]:
### Write your code here
book_43_counts_filtered.count()
print(f"Book 43: {book_84_counts_filtered.count()}")

Book 43: 7016


### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [110]:
### Write your code here
def process_RDD(filepath):
    # Read in the file as a textRDD
    book = sc.textFile(filepath)
    # Clean and split the line using clean_split_line
    filtered_words = book.flatMap(clean_split_line)
    # Filters out the potential stop words
    counts = filtered_words.map(lambda x: (x.lower(), 1)).reduceByKey(lambda x, y: x+y)
    new_book_count = counts.filter(lambda x: x[0] not in list_stops_words)
    # Returns the word count RDD as a tuple
    return new_book_count
# Test to see if the code was run...
print("Function process_RDD complete!")

Function process_RDD complete!


### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [148]:
### Write your code here
# Reading in the files
book_84_counts_filtered = process_RDD('data/84.txt')
book_398_counts_filtered = process_RDD('data/398.txt')
book_3296_counts_filtered = process_RDD('data/3296.txt')

In [114]:
# Printing out the number of distinct words
print(f"Book 84: {book_84_counts_filtered.count()}")
print(f"Book 398: {book_398_counts_filtered.count()}")
print(f"Book 3296: {book_3296_counts_filtered.count()}")

Book 84: 7016
Book 398: 2421
Book 3296: 7293


### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [122]:
### Write your code here
joined_398_3296 = book_398_counts_filtered.join(book_3296_counts_filtered).count()
joined_84_398 = book_84_counts_filtered.join(book_398_counts_filtered).count()

In [123]:
print(f"Book 398 and Book 3296: {joined_398_3296}")
print(f"Book 84 and Book 398: {joined_84_398}")

Book 398 and Book 3296: 1790
Book 84 and Book 398: 1691


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

In [125]:
# Check for the commonality between book_84 and book_3296
joined_84_3296 = book_84_counts_filtered.join(book_3296_counts_filtered).count()
print(f"Book 84 and Book 3296: {joined_84_3296}")

Book 84 and Book 3296: 3608


In [131]:
# Percentage Difference
diff = (joined_398_3296 + joined_84_398)/joined_84_3296
diff

0.9648004434589801

#### Answer

Based on the above, counting the number of shared words is not a good idea as a distance metric for evaluating topic similarity. This is justified because although both of the books within the metric share the topic of religion, counting the number of shared words between books does not necessarily mean topic similarity exists if there are a large number of words in the book in comparison to another with a smaller number. Book_84 and Book_3296 however, have a larger sharing of words, but their commonality in type is not the same. This means that calculating the number of shared words is not a good metric, because shared words do not equate to similar topic. 

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [161]:
### Write your code here
import numpy as np

book_43_counts_filtered = process_RDD("data/43.txt")
book_84_counts_filtered = process_RDD("data/84.txt")
book_398_counts_filtered = process_RDD("data/398.txt")
book_3296_counts_filtered = process_RDD("data/3296.txt")

In [167]:
# Implementation of multiple joins 
all_joined = book_43_counts_filtered.join(book_84_counts_filtered).join(book_398_counts_filtered).join(book_3296_counts_filtered)
all_join_count = all_joined.mapValues(lambda x: (x[0][0] + (x[0][1],) + (x[1],)))
counts_matrix = np.zeros([4,all_joined.count()])

counts_matrix[0, :] = all_joined.map(lambda x: x[1][0][0][0]).collect()
counts_matrix[1, :] = all_joined.map(lambda x: x[1][0][0][1]).collect()
counts_matrix[2, :] = all_joined.map(lambda x: x[1][0][1]).collect()
counts_matrix[3, :] = all_joined.map(lambda x: x[1][1]).collect()

counts_matrix

array([[88., 98., 13., ...,  1.,  1.,  1.],
       [88., 98., 13., ...,  2.,  2.,  1.],
       [87., 93.,  9., ...,  1.,  1.,  1.],
       [88., 97., 10., ...,  1.,  1.,  1.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [168]:
### Write your code here
from scipy.spatial.distance import euclidean

In [169]:
print(f"Book 398 & Book 3926: {euclidean(counts_matrix[2], counts_matrix[3])}")
print(f"Book 84 & Book 398: {euclidean(counts_matrix[1], counts_matrix[2])}")

Book 398 & Book 3926: 1156.6628722320086
Book 84 & Book 398: 751.6688100486809


#### Justification
In conclusion to Euclidian distance and the evaluation of topic relatedness across documents, it can be understood that this is not the optimal evaluative method to determine whether topics are related. This is because the Euclidian distance between books that are supposed to be similar (398 & 3926) had a far grater distance in comprison to books 84 & 398. This denotes that the distance between these two are uncorrelated in terms of topics. 

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Answer
In reference to the most recent class (11/08/21), a good way to improve the similarity between documents that pertain to the same topic is to account for the format of the words and their vectors. By applying a dictionary of keys to words and the similarities in how their predispositioned elements are in comparison to each other, there is a far greater chance of the meaning (context) of the topics and text being similar as oppose to simply counting frequency, or calculating Euclidian distance. 

For example, calculating the order of which the words are within text and mapping each of these words to a key value. If the values have a corresponding match, it is said to believe that the context and topics within these texts are similar. 

## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [22]:
### Write your code here
from pyspark.sql import SparkSession

In [25]:
ss = SparkSession(sc)
type(ss)

pyspark.sql.session.SparkSession

In [29]:
#Loads the flight_info.csv into the Spark DataFrame
flight_info = ss.read.csv("data/flight_info.csv", header=True)
flight_info.head(5)

[Row(_c0='0', DayOfWeek='2', UniqueCarrier='AA', FlightNum='494', Origin='CLT', Dest='PHX', CRSDepTime='1619', DepTime='1616.0', TaxiOut='17.0', WheelsOff='1633.0', WheelsOn='1837.0', TaxiIn='5.0', CRSArrTime='1856', ArrTime='1842.0', Cancelled='0.0', CancellationCode=None, Distance='1773.0', CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None),
 Row(_c0='1', DayOfWeek='3', UniqueCarrier='AA', FlightNum='494', Origin='CLT', Dest='PHX', CRSDepTime='1619', DepTime='1614.0', TaxiOut='13.0', WheelsOff='1627.0', WheelsOn='1815.0', TaxiIn='6.0', CRSArrTime='1856', ArrTime='1821.0', Cancelled='0.0', CancellationCode=None, Distance='1773.0', CarrierDelay=None, WeatherDelay=None, NASDelay=None, SecurityDelay=None, LateAircraftDelay=None),
 Row(_c0='2', DayOfWeek='4', UniqueCarrier='AA', FlightNum='494', Origin='CLT', Dest='PHX', CRSDepTime='1619', DepTime='1611.0', TaxiOut='17.0', WheelsOff='1628.0', WheelsOn='1824.0', TaxiIn='2.0', CRSArrTime='1856',

In [32]:
print(f"The number of entries contained in this file are: {flight_info.count()}")

The number of entries contained in this file are: 450017


### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [38]:
### Write your code here
flight_info.createOrReplaceTempView("flights")
unique_flights = ss.sql(
"""
SELECT DISTINCT UniqueCarrier FROM flights
"""
).show()

+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|           EV|
|           B6|
|           DL|
|           UA|
|           NK|
|           OO|
|           F9|
|           HA|
|           WN|
|           AS|
|           VX|
+-------------+



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [76]:
### Write your code here
delays = ss.sql(
"""
SELECT UniqueCarrier, count(*) AS Delayed
FROM flights
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier
ORDER BY Delayed DESC
"""
).show()

+-------------+-------+
|UniqueCarrier|Delayed|
+-------------+-------+
|           WN|  54557|
|           DL|  28962|
|           AA|  26291|
|           UA|  19594|
|           OO|  17924|
|           EV|  12340|
|           B6|  10406|
|           AS|   4966|
|           NK|   4435|
|           F9|   3181|
|           VX|   2871|
|           HA|   2265|
+-------------+-------+



### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [154]:
### Write your code here
airlines_info = ss.read.csv("data/airlines.csv", header=False)
airlines_info.show(5)

+---+--------------------+---+----+---+-------+--------------+---+
|_c0|                 _c1|_c2| _c3|_c4|    _c5|           _c6|_c7|
+---+--------------------+---+----+---+-------+--------------+---+
|  1|      Private flight| \N|   -|N/A|   null|          null|  Y|
|  2|         135 Airways| \N|null|GNL|GENERAL| United States|  N|
|  3|       1Time Airline| \N|  1T|RNX|NEXTIME|  South Africa|  Y|
|  4|2 Sqn No 1 Elemen...| \N|null|WYT|   null|United Kingdom|  N|
|  5|     213 Flight Unit| \N|null|TFU|   null|        Russia|  N|
+---+--------------------+---+----+---+-------+--------------+---+
only showing top 5 rows



In [159]:
airlines_info.createOrReplaceTempView("airlines")

### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [75]:
### Write your code here
delays_per_day = ss.sql(
"""
SELECT UniqueCarrier, DayOfWeek, COUNT(*) as Delayed
FROM flights
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier, DayOfWeek
ORDER BY DayOfWeek ASC, UniqueCarrier ASC
"""
).show()

+-------------+---------+-------+
|UniqueCarrier|DayOfWeek|Delayed|
+-------------+---------+-------+
|           AA|        1|   5117|
|           AS|        1|    872|
|           B6|        1|   1892|
|           DL|        1|   6292|
|           EV|        1|   2605|
|           F9|        1|    632|
|           HA|        1|    300|
|           NK|        1|    848|
|           OO|        1|   3467|
|           UA|        1|   3893|
|           VX|        1|    564|
|           WN|        1|   9970|
|           AA|        2|   3688|
|           AS|        2|    669|
|           B6|        2|   1787|
|           DL|        2|   4008|
|           EV|        2|   2032|
|           F9|        2|    494|
|           HA|        2|    336|
|           NK|        2|    667|
+-------------+---------+-------+
only showing top 20 rows



### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [134]:
### Write your code here
# Get the total flights
total_flights = ss.sql(
"""
SELECT UniqueCarrier, COUNT(*) AS Delayed
FROM flights
GROUP BY UniqueCarrier
"""
).show()

+-------------+-------+
|UniqueCarrier|Delayed|
+-------------+-------+
|           AA|  73132|
|           EV|  35037|
|           B6|  24602|
|           DL|  69813|
|           UA|  42403|
|           NK|  12570|
|           OO|  50146|
|           F9|   7760|
|           HA|   6276|
|           WN| 107785|
|           AS|  14711|
|           VX|   5782|
+-------------+-------+



In [ ]:
# Normalize the Query
normalized_flights = ss.sql(
"""

"""
)

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [170]:
### Write your code here
%%time

UsageError: Line magic function `%%time` not found.


### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [ ]:
### Write your code here

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [ ]:
### Write your code here